# Generalized Assignment Problem
Pada sesi ini kita akan menggunakan library pulp untuk menyelesaikan Generalized Assignment Problem. GAP memiliki tujuan meminimalkan biaya atau memaksimalkan keuntungan dengan cara menugaskan n tugas pada m server yang memiliki kapasitas terbatas serta **setiap tugas hanya boleh dikerjakan oleh satu server**. Untuk lebih lengkapnya ada di buku **Sarker and Newton - Optimization Modelling, A Practical Approach**

**Berikut adalah parameter yang digunakan:**

- n = jumlah tugas/pekerjaan
- m = jumlah server/pekerja
- Cij = biaya untuk menugaskan pekerjaan i ke server j
- bj = kapasitas/sumber daya yang tersedia pada server j
- aij = jumlah sumber daya yang diperlukan untuk melakukan pekerjaan i oleh server j

Sedangkan variabel yang digunakan yakni **Xij** yang bertipe binary (1 = dilakukan, 0 = tidak dilakukan)

**Berikut adalah fungsi tujuan beserta constrain:**
<img src="https://user-images.githubusercontent.com/61647791/128858690-a3d696fb-7b84-42c1-bdc6-15a07fe0291f.PNG" />
*Sumber: Sarker and Newton - Optimization Modelling, A Practical Approach*


**Batasan yang harus dipenuhi yakni:**
1. Setiap tugas/pekerjaan hanya boleh dikerjakan oleh satu server
2. Jumlah sumber daya yang digunakan vendor pada beberapa pekerjaan harus kurang dari atau sama dengan kapasitasnya

**Ilustrasi Permasalahan:**

Sebuah instansi akan mengadakan acara perayaan dengan menyediakan 9 jenis makanan (data menu). Instansi tersebut memiliki data nama-nama vendor (vendor), biaya pemesanan makanan (data costs) serta kapasitas jumlah karyawan pada masing-masing vendor dan menu (resources). Bagaimanakah alokasi vendor tersebut supaya biaya yang dikeluarkan instansi bisa seminimal mungkin.

*Data yang digunakan bersumber dari mata kuliah yang saya pelajari, Matematika Optimasi dengan Bpk 
Setyo Tri Windras Mara*. Pada mata kuliah tersebut saya menyelesaikan permasalahan ini menggunakan library python Gurobi. Namun kali ini saya akan menyelesaikannya menggunakan library PuLP.

### 1. Mengimpor Library

In [1]:
from pulp import *
import pandas as pd
import numpy as np

### 2. Mengimpor Data
Data yang digunakan memiliki format .xlsx dengan 4 sheet. Saya menggunakan tambahan parameter **header** dan **usecols** untuk merapikan tabel tersebut. Saya juga menyamakan indeks untuk kemudahan akses pada tahap selanjutnya. Dapat dilihat bentuk masing-masing tabel yang akan digunakan. 

In [5]:
#load data
menu = pd.read_excel("Generalized Assignment Problem.xlsx", sheet_name="Menu", header=0).set_index('Menu')
vendor = pd.read_excel("Generalized Assignment Problem.xlsx", sheet_name="Vendor", header=0).set_index('Vendor')
costs = pd.read_excel("Generalized Assignment Problem.xlsx", sheet_name="Biaya", header=1,usecols=[2,3,4,5,6])
resources = pd.read_excel("Generalized Assignment Problem.xlsx", sheet_name="Resources", header=1,usecols=[2,3,4,5,6])

#menyamakan index
costs['menu'] = menu.index
costs.set_index('menu', inplace=True)

resources['menu'] = menu.index
resources.set_index('menu', inplace=True)

#### a. Menu
Tabel menu hanya terdiri dari indeks menu beserta nama menu

In [6]:
menu

,Nama Menu
Menu,
1,Bakso
2,Soto
3,Nasi Sayur
4,Empal Gentong
5,Zuppa Soup
6,Kambing Guling
7,Selat Solo
8,Snack
9,Bebek Peking


#### b. Vendor
Tabel vendor berisi indeks vendor, nama vendor dan jumlah karyawan yang dimiliki.

In [7]:
vendor

,Nama Vendor,Jumlah Karyawan
Vendor,,
1,Sami Asih Group,20
2,Sumber Vitamin,30
3,Katineung,20
4,Murni,15
5,Sangkuriang,15


#### c. Costs
Tabel costs berisi biaya yang diperlukan pada masing-masing menu beserta vendornya

In [96]:
costs

,1,2,3,4,5
menu,,,,,
1,197,114,173,175,137
2,184,108,148,188,147
3,151,156,122,124,118
4,200,109,111,132,157
5,110,130,142,129,182
6,171,159,200,171,125
7,114,200,112,185,138
8,150,165,156,136,100
9,184,129,193,199,148


#### d. Resources
Tabel ini berisi jumlah karyawan yang dibutuhkan pada masing-masing vendor dan menu

In [97]:
resources

,1,2,3,4,5
menu,,,,,
1,5,5,2,5,2
2,2,4,5,1,5
3,5,1,4,4,1
4,5,3,4,1,4
5,5,5,2,3,4
6,3,4,4,3,2
7,4,5,2,2,3
8,2,5,4,3,2
9,3,4,4,5,3


### 3. Menentukan Variabel
Di samping membuat variabel yang digunakan, saya juga membuat beberapa parameter untuk kemudahan akses data

In [25]:
#membuat parameter
list_vendor = vendor.index
list_menu = menu.index
keys = [(i,j) for i in list_menu for j in list_vendor]

#variable
x = LpVariable.dicts("x", keys, 0, None, 'Binary')

### 4. Menginisiasi Model
Permasalahan ini memiliki tujuan meminimalkan biaya sehingga kita menggunakan **LpMinimize**

In [40]:
model = LpProblem('Choosing_Vendor', LpMinimize)

### 5. Menambahkan Fungsi Tujuan dan Constrain

In [41]:
#fungsi tujuan memimalkan biaya Cij ke Xij
model += lpSum([costs.loc[i,j]*x[i,j] for i in list_menu for j in list_vendor])

In [42]:
#constrain

#setiap pekerjaan hanya boleh dilakukan oleh satu vendor
for i in list_menu:
    model += lpSum([x[i,j] for j in list_vendor]) == 1
    
#jumlah karyawan yang digunakan vendor pada beberapa pekerjaan harus kurang dari atau sama dengan kapasitas karyawan di vendor   
for j in list_vendor:
    model += lpSum([resources.loc[i,j]*x[i,j] for i in list_menu] ) <= vendor.loc[j,'Jumlah Karyawan']

### 6. Menyelesaikan Model serta Mengecek Hasil Optimasi

In [43]:
#apabila status model sama dengan 1 artinya model sudah optimal
model.solve()

1

In [45]:
#cek biaya yang harus dikeluarkan instansi
model.objective.value()

1025

In [46]:
#membuat list yang berisikan solusi 
solutions = []
for i,j in x.keys():
    if (x[i,j].varValue) == 1:
        solutions.append(x[i,j])

In [47]:
solutions

[x_(1,_2),
 x_(2,_2),
 x_(3,_5),
 x_(4,_2),
 x_(5,_1),
 x_(6,_5),
 x_(7,_3),
 x_(8,_5),
 x_(9,_2)]

Dari list solusi di atas angka di depan menunjukkan indeks menu kemudian diikuti indeks vendor. Terlihat bahwa bentuknya masih kurang bagus sehingga perlu dilakukan sedikit modifikasi untuk mengakses angkanya saja.

In [48]:
vendor_sol = [int(str(solutions[j])[6]) for j in range(0,9)]
vendor_sol

[2, 2, 5, 2, 1, 5, 3, 5, 2]

Saya juga membuat dictionary pada menu dan vendor untuk membuat tabel yang mudah dipahami 

In [35]:
dict_menu = {i:j for (i,j) in zip(menu.index, list(menu['Nama Menu']))}
dict_menu

{1: 'Bakso',
 2: 'Soto',
 3: 'Nasi Sayur',
 4: 'Empal Gentong',
 5: 'Zuppa Soup',
 6: 'Kambing Guling',
 7: 'Selat Solo',
 8: 'Snack',
 9: 'Bebek Peking'}

In [36]:
dict_vendor = {i:j for (i,j) in zip(vendor.index, list(vendor['Nama Vendor']))}
dict_vendor

{1: 'Sami Asih Group',
 2: 'Sumber Vitamin',
 3: 'Katineung',
 4: 'Murni',
 5: 'Sangkuriang'}

In [49]:
#membuat tabel -> sebelum diubah
df_sol = pd.DataFrame({'Menu': list_menu,
                       'Vendor':vendor_sol})

df_sol

,Menu,Vendor
0,1,2
1,2,2
2,3,5
3,4,2
4,5,1
5,6,5
6,7,3
7,8,5
8,9,2


Tabel di atas masih berupa angka saja sehingga kurang bermakna. Oleh karena itu saya melakukan koversi angka tersebut menggunakan dictionary yang sudah saya buat di atas.

In [51]:
df_sol['Menu'] = df_sol["Menu"].replace(dict_menu)
df_sol['Vendor'] = df_sol["Vendor"].replace(dict_vendor)

df_sol

,Menu,Vendor
0,Bakso,Sumber Vitamin
1,Soto,Sumber Vitamin
2,Nasi Sayur,Sangkuriang
3,Empal Gentong,Sumber Vitamin
4,Zuppa Soup,Sami Asih Group
5,Kambing Guling,Sangkuriang
6,Selat Solo,Katineung
7,Snack,Sangkuriang
8,Bebek Peking,Sumber Vitamin


Tabel di atas sudah lebih mudah dipahami sehingga bisa langsung kita save/export dalam bentuk file excel.

In [38]:
df_sol.to_excel("Choosing_Vendor.xlsx")

Kita juga dapat melakukan groupby pada vendor untuk melihat vendor mana yang paling banyak dipilih oleh instansi. Dari hasil groupby vendor Sumber Vitamin paling banyak dipilih/diandalkan untuk pemesanan dan pelayanan makanan.

In [132]:
df_sol.groupby('Vendor').count()

,Menu
Vendor,
Katineung,1
Sami Asih Group,1
Sangkuriang,3
Sumber Vitamin,4


### Penutup
Demikian tutorial singkat menyelesaikan permasalahan GAP menggunakan pulp. Mohon maaf jika terdapat kekurangan, sekian dan terima kasih. Selamat mencoba :)